# Supply Chain End-to-End

## Overview

Complete supply chain intelligence: multi-source data ingestion, build supply chain knowledge graph, analyze dependencies, optimize flow, and predict disruptions.

## Workflow: Multi-Source Data → Build Supply Chain KG → Analyze Dependencies → Optimize → Predict Disruptions


In [ ]:
from semantica.ingest import FileIngestor, WebIngestor, DBIngestor, StreamIngestor
from semantica.parse import DocumentParser, WebParser, StructuredDataParser
from semantica.semantic_extract import NERExtractor, RelationExtractor
from semantica.kg import GraphBuilder, GraphAnalyzer, ConnectivityAnalyzer, TemporalPatternDetector
from semantica.reasoning import InferenceEngine
from datetime import datetime, timedelta
import os
import tempfile
import json


## Step 1: Multi-Source Data Ingestion


In [ ]:
file_ingestor = FileIngestor()
web_ingestor = WebIngestor()
db_ingestor = DBIngestor()
stream_ingestor = StreamIngestor()
document_parser = DocumentParser()
web_parser = WebParser()
structured_parser = StructuredDataParser()

temp_dir = tempfile.mkdtemp()

report_file = os.path.join(temp_dir, "supply_chain_report.txt")
with open(report_file, 'w') as f:
    f.write("Supplier A delivers components to Factory B. Supplier C supplies Factory D.")

file_objects = file_ingestor.ingest_file(report_file, read_content=True)
parsed_file_content = document_parser.extract_text(report_file) if file_objects else ""

web_content = "Shipping delays reported in region X due to weather conditions."
parsed_web_content = web_parser.parse_text(web_content) if web_content else ""

db_data_file = os.path.join(temp_dir, "suppliers_db.json")
db_data = [
    {"supplier": "Supplier A", "factory": "Factory B", "status": "active"},
    {"supplier": "Supplier C", "factory": "Factory D", "status": "active"}
]
with open(db_data_file, 'w') as f:
    json.dump(db_data, f)

parsed_db_data = structured_parser.parse_json(db_data_file)

stream_events = [
    {"event": "shipment_delayed", "supplier": "Supplier A", "timestamp": datetime.now().isoformat()}
]

all_data = []
if parsed_file_content:
    all_data.append({"source": "file", "content": parsed_file_content, "type": "report"})
if parsed_web_content:
    all_data.append({"source": "web", "content": parsed_web_content, "type": "news"})
for db_record in parsed_db_data.get("data", db_data):
    all_data.append({"source": "db", **db_record})
for stream_event in stream_events:
    all_data.append({"source": "stream", **stream_event})

print(f"Ingested data from {len(set(d.get('source') for d in all_data))} sources")
print(f"  File sources: {len([d for d in all_data if d.get('source') == 'file'])}")
print(f"  Web sources: {len([d for d in all_data if d.get('source') == 'web'])}")
print(f"  Database sources: {len([d for d in all_data if d.get('source') == 'db'])}")
print(f"  Stream sources: {len([d for d in all_data if d.get('source') == 'stream'])}")
print(f"Total data items: {len(all_data)}")


## Step 2: Build Supply Chain Knowledge Graph


In [ ]:
ner_extractor = NERExtractor()
relation_extractor = RelationExtractor()
builder = GraphBuilder()

supply_chain_entities = []
relationships = []
entity_map = {}

for data_item in all_data:
    content = data_item.get("content", "")
    if not content:
        content = str(data_item)
    
    extracted_entities = ner_extractor.extract(content)
    extracted_relations = relation_extractor.extract(content, extracted_entities)
    
    for entity in extracted_entities:
        entity_text = entity.get("text", entity.get("entity", ""))
        entity_type = entity.get("type", "Entity")
        
        if entity_text and entity_text not in entity_map:
            entity_id = entity_text.lower().replace(" ", "_")
            entity_map[entity_text] = entity_id
            
            if "supplier" in entity_text.lower() or "supplier" in entity_type.lower():
                entity_type = "Supplier"
            elif "factory" in entity_text.lower() or "factory" in entity_type.lower():
                entity_type = "Factory"
            elif "warehouse" in entity_text.lower():
                entity_type = "Warehouse"
            elif "product" in entity_text.lower():
                entity_type = "Product"
            
            supply_chain_entities.append({
                "id": entity_id,
                "type": entity_type,
                "name": entity_text,
                "properties": {}
            })
    
    for rel in extracted_relations:
        source_text = rel.get("source", "")
        target_text = rel.get("target", "")
        rel_type = rel.get("type", "related_to")
        
        if source_text in entity_map and target_text in entity_map:
            relationships.append({
                "source": entity_map[source_text],
                "target": entity_map[target_text],
                "type": rel_type,
                "properties": {"timestamp": datetime.now().isoformat()}
            })

for db_record in parsed_db_data.get("data", db_data):
    supplier_name = db_record.get("supplier", "")
    factory_name = db_record.get("factory", "")
    
    if supplier_name and factory_name:
        supplier_id = supplier_name.lower().replace(" ", "_")
        factory_id = factory_name.lower().replace(" ", "_")
        
        if supplier_id not in entity_map:
            entity_map[supplier_name] = supplier_id
            supply_chain_entities.append({
                "id": supplier_id,
                "type": "Supplier",
                "name": supplier_name,
                "properties": {}
            })
        
        if factory_id not in entity_map:
            entity_map[factory_name] = factory_id
            supply_chain_entities.append({
                "id": factory_id,
                "type": "Factory",
                "name": factory_name,
                "properties": {"capacity": 1000}
            })
        
        relationships.append({
            "source": supplier_id,
            "target": factory_id,
            "type": "supplies",
            "properties": {"timestamp": datetime.now().isoformat(), "status": "active"}
        })

if "warehouse_1" not in entity_map:
    supply_chain_entities.append({
        "id": "warehouse_1",
        "type": "Warehouse",
        "name": "Warehouse 1",
        "properties": {}
    })
    entity_map["Warehouse 1"] = "warehouse_1"

if "factory_b" in entity_map and "warehouse_1" in entity_map:
    relationships.append({
        "source": "factory_b",
        "target": "warehouse_1",
        "type": "ships_to",
        "properties": {"timestamp": datetime.now().isoformat()}
    })

supply_chain_kg = builder.build(supply_chain_entities, relationships)

print(f"Extracted {len([e for e in supply_chain_entities if e['type'] in ['Supplier', 'Factory']])} supply chain entities from parsed data")
print(f"Built supply chain knowledge graph with {len(supply_chain_entities)} entities and {len(relationships)} relationships")


## Step 3: Analyze Dependencies


In [ ]:
analyzer = GraphAnalyzer()
connectivity_analyzer = ConnectivityAnalyzer()

connectivity_analysis = connectivity_analyzer.analyze_connectivity(supply_chain_kg)
graph_metrics = analyzer.compute_metrics(supply_chain_kg)

entities_list = supply_chain_kg.get("entities", [])
relationships_list = supply_chain_kg.get("relationships", [])
entity_map = {e.get("id"): e for e in entities_list}

dependencies = []
for entity in entities_list:
    entity_id = entity.get("id")
    incoming = [r for r in relationships_list if r.get("target") == entity_id]
    outgoing = [r for r in relationships_list if r.get("source") == entity_id]
    
    if incoming or outgoing:
        dependencies.append({
            "entity_id": entity_id,
            "entity_type": entity.get("type"),
            "name": entity.get("name"),
            "incoming_dependencies": len(incoming),
            "outgoing_dependencies": len(outgoing),
            "depends_on": [entity_map.get(r.get("source"), {}).get("name", r.get("source")) for r in incoming if entity_map.get(r.get("source"))],
            "supports": [entity_map.get(r.get("target"), {}).get("name", r.get("target")) for r in outgoing if entity_map.get(r.get("target"))]
        })

print(f"Analyzed dependencies for {len(dependencies)} entities")
print(f"Graph connectivity: {connectivity_analysis.get('is_connected', False)}")
print(f"Connected components: {len(connectivity_analysis.get('components', []))}")
for dep in dependencies:
    print(f"  {dep['name']} ({dep['entity_type']}): {dep['incoming_dependencies']} incoming, {dep['outgoing_dependencies']} outgoing")


## Step 4: Optimize Flow


In [ ]:
inference_engine = InferenceEngine()

inference_engine.add_rule("IF factory has less than 2 suppliers THEN suggest add_redundancy")
inference_engine.add_rule("IF entity has no incoming dependencies AND has more than 2 outgoing THEN suggest bottleneck_mitigation")

optimization_facts = []
for dep in dependencies:
    if dep["entity_type"] == "Factory" and dep["incoming_dependencies"] < 2:
        optimization_facts.append({
            "entity": dep["entity_id"],
            "type": "factory",
            "supplier_count": dep["incoming_dependencies"]
        })
    if dep["incoming_dependencies"] == 0 and dep["outgoing_dependencies"] > 2:
        optimization_facts.append({
            "entity": dep["entity_id"],
            "type": "bottleneck",
            "outgoing_count": dep["outgoing_dependencies"]
        })

if optimization_facts:
    inference_engine.add_facts(optimization_facts)
    inferred_results = inference_engine.forward_chain()
else:
    inferred_results = []

optimized_flow = []
factories = [e for e in entities_list if e.get("type") == "Factory"]
for factory in factories:
    factory_id = factory.get("id")
    incoming = [r for r in relationships_list if r.get("target") == factory_id and r.get("type") == "supplies"]
    if len(incoming) < 2:
        optimized_flow.append({
            "type": "add_redundancy",
            "entity": factory.get("name"),
            "suggestion": f"Add backup supplier for {factory.get('name')} to reduce risk"
        })

bottlenecks = [d for d in dependencies if d["incoming_dependencies"] == 0 and d["outgoing_dependencies"] > 2]
if bottlenecks:
    optimized_flow.append({
        "type": "bottleneck_detected",
        "entities": [b["name"] for b in bottlenecks],
        "suggestion": "Consider adding parallel paths for critical nodes"
    })

if inferred_results:
    print(f"Inference engine generated {len(inferred_results)} optimization inferences")

print(f"Generated {len(optimized_flow)} optimization suggestions")
for suggestion in optimized_flow:
    print(f"  {suggestion['type']}: {suggestion['suggestion']}")


## Step 5: Predict Disruptions


In [ ]:
pattern_detector = TemporalPatternDetector()

temporal_patterns = pattern_detector.detect_temporal_patterns(
    supply_chain_kg,
    pattern_type="anomaly",
    min_frequency=1
)

disruptions = []

delay_events = [d for d in all_data if d.get("event") == "shipment_delayed" or "delay" in str(d.get("content", "")).lower()]

if delay_events:
    disruptions.append({
        "type": "delivery_delay",
        "severity": "high",
        "affected_entities": ["Supplier A"],
        "description": "Shipping delays detected in supply chain",
        "recommendation": "Activate backup suppliers or adjust production schedules"
    })

single_supplier_factories = []
for factory in [e for e in supply_chain_kg.get("entities", []) if e.get("type") == "Factory"]:
    factory_id = factory.get("id")
    suppliers = [r for r in supply_chain_kg.get("relationships", []) if r.get("target") == factory_id and r.get("type") == "supplies"]
    if len(suppliers) == 1:
        single_supplier_factories.append(factory.get("name"))

if single_supplier_factories:
    disruptions.append({
        "type": "single_point_of_failure",
        "severity": "medium",
        "affected_entities": single_supplier_factories,
        "description": "Factories with single supplier dependency detected",
        "recommendation": "Add redundant supplier relationships"
    })

if temporal_patterns:
    disruptions.append({
        "type": "temporal_anomaly",
        "severity": "medium",
        "description": f"Detected {len(temporal_patterns)} temporal anomalies in supply chain",
        "recommendation": "Review temporal patterns for potential disruptions"
    })

print(f"Predicted {len(disruptions)} potential disruptions")
for disruption in disruptions:
    print(f"  {disruption['type']} ({disruption['severity']}): {disruption['description']}")
    print(f"    Recommendation: {disruption['recommendation']}")

entities_count = len(supply_chain_kg.get("entities", []))
print(f"\nAnalyzed {entities_count} supply chain nodes")


## Summary

Complete supply chain intelligence workflow:
- Multi-source data ingested
- Supply chain knowledge graph built
- Dependencies analyzed
- Flow optimization suggested
- Disruptions predicted
